### caffe gen a conv layer

In [102]:
import sys

In [103]:
sys.path.insert(0, '/home/niekai/caffe/python/')

In [104]:
import numpy as np

import caffe

import torch

import mxnet as mx

In [105]:
from caffe.proto import caffe_pb2

import google.protobuf as pb

from caffe import to_proto

from caffe import params as P

from caffe import layers as L

In [106]:
from google.protobuf import text_format

#### Define gen conv layer

In [107]:
def _get_param(num_param):
    if num_param==1:
        #only_weight
        param = caffe_pb2.ParamSpec()
        param.lr_mult = 1
        param.decay_mult = 1
        return [param]
    if num_param==2:
        #weight and bias
        param_w = caffe_pb2.ParamSpec()
        param_w.lr_mult = 1
        param_w.decay_mult = 1
        param_b = caffe_pb2.ParamSpec()
        param_b.lr_mult = 2
        param_b.decay_mult = 0
        return [param_w, param_b]
    else:
        raise ValueError("Unknown num_param {}".format(num_param))

In [108]:
def gen_conv_layer(name, bottom, top, num_output, kernel_size, stride, pad):
    conv_layer = caffe_pb2.LayerParameter()
    conv_layer.name = name
    conv_layer.type = 'Convolution'
    conv_layer.bottom.extend([bottom])
    conv_layer.top.extend([top])
    conv_layer.convolution_param.num_output = num_output
    conv_layer.convolution_param.kernel_size.extend([kernel_size])
    conv_layer.convolution_param.stride.extend([stride])
    conv_layer.convolution_param.pad.extend([pad])
    conv_layer.convolution_param.weight_filler.type='msra'
    conv_layer.convolution_param.bias_term=False
    conv_layer.param.extend(_get_param(1))
    return conv_layer

#### Generate conv layer

- num_output = 16

In [155]:
conv_layer = gen_conv_layer(name='conv0', bottom='data', top='conv0',
                           num_output=16, kernel_size=3, stride=1, pad=1)

In [156]:
layers = []
layers.append(conv_layer)

#### Define model

In [157]:
model = caffe_pb2.NetParameter()

In [158]:
model.name = 'test_conv_net'
model.layer.extend(layers)
model.input.append('data')
model.input_dim.append(1)
model.input_dim.append(3)
model.input_dim.append(10)
model.input_dim.append(10)

#### Save model to prototxt

In [159]:
with open('deploy_test_conv_net.prototxt', 'w') as f:
    f.write(text_format.MessageToString(model))

#### load prototxt and fill val of param

In [160]:
net = caffe.Net('deploy_test_conv_net.prototxt', caffe.TEST)

In [161]:
net.blobs['data'].data[...] = np.ones(shape=(1,3,10,10))

In [162]:
out = net.forward()

In [163]:
out['conv0'].shape

(1, 16, 10, 10)

In [164]:
net.save('deploy_test_conv_net.caffemodel')

#### load prototxt and caffemodel

In [176]:
net = caffe.Net('deploy_test_conv_net.prototxt', 'deploy_test_conv_net.caffemodel',
               caffe.TEST)

In [177]:
net.blobs['data'].data[...] = np.ones(shape=(1,3,10,10))

In [178]:
out = net.forward()

In [179]:
out['conv0'].shape

(1, 16, 10, 10)

In [180]:
out['conv0']

array([[[[-4.0441281e-01, -3.2275343e-01, -3.2275343e-01, ...,
          -3.2275343e-01, -3.2275343e-01, -9.0664327e-01],
         [-4.6315241e-01, -4.7584307e-01, -4.7584307e-01, ...,
          -4.7584307e-01, -4.7584307e-01, -5.0296056e-01],
         [-4.6315241e-01, -4.7584307e-01, -4.7584307e-01, ...,
          -4.7584307e-01, -4.7584307e-01, -5.0296056e-01],
         ...,
         [-4.6315241e-01, -4.7584307e-01, -4.7584307e-01, ...,
          -4.7584307e-01, -4.7584307e-01, -5.0296056e-01],
         [-4.6315241e-01, -4.7584307e-01, -4.7584307e-01, ...,
          -4.7584307e-01, -4.7584307e-01, -5.0296056e-01],
         [ 2.7718768e-02, -3.0732444e-01, -3.0732444e-01, ...,
          -3.0732444e-01, -3.0732444e-01, -2.5161436e-01]],

        [[-9.4936836e-01, -1.7612537e+00, -1.7612537e+00, ...,
          -1.7612537e+00, -1.7612537e+00, -7.6124907e-01],
         [-7.5664794e-01, -1.3808473e+00, -1.3808473e+00, ...,
          -1.3808473e+00, -1.3808473e+00, -5.5388826e-01],
        